## Work
### 請結合前面的知識與程式碼，比較不同的 regularization 的組合對訓練的結果與影響：如 dropout, regularizers, batch-normalization 等

In [1]:
import os
import keras
import itertools
# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.regularizers import l1,l2,l1_l2

def build_mlp(input_shape,num_neurons=[512,256,128],output=10):
    input_layer = keras.layers.Input(input_shape)
    x = BatchNormalization()(input_layer)
    for i,neurons in enumerate(num_neurons):
        x = keras.layers.Dense(units=neurons,activation='relu',kernel_regularizer=l1(1e-4),name='hidden_layer_'+str(i+1))(x)
        
    x =Dropout(0.1)(x)    
    out = keras.layers.Dense(units=output,activation='softmax',name='output_layer')(x)
    
    model = keras.models.Model(inputs=[input_layer],outputs=[out])
            
    
    """Code Here
    建立你的神經網路
    """
    return model

In [6]:

"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
MOMENTUM = 0.95


In [9]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""

model = build_mlp(x_train.shape[1:])
opt= keras.optimizers.SGD(lr = LEARNING_RATE)

model.compile(loss = 'categorical_crossentropy',optimizer = opt,metrics=['accuracy'])
model.fit(x_train,y_train,
         epochs = EPOCHS,
         batch_size= 512,
         validation_data = (x_test,y_test),
         shuffle=True)

train_loss = model.history.history['loss']
valid_loss = model.history.history['val_loss']
train_acc = model.history.history['acc']
val_acc = model.history.history['val_acc']

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 4s 80us/step - loss: 6.4588 - acc: 0.1399 - val_loss: 6.2386 - val_acc: 0.1926
Epoch 2/50
50000/50000 [==============================] - 3s 63us/step - loss: 6.1976 - acc: 0.2081 - val_loss: 6.1176 - val_acc: 0.2368
Epoch 3/50
50000/50000 [==============================] - 3s 64us/step - loss: 6.1090 - acc: 0.2398 - val_loss: 6.0498 - val_acc: 0.2614
Epoch 4/50
50000/50000 [==============================] - 3s 59us/step - loss: 6.0520 - acc: 0.2621 - val_loss: 6.0020 - val_acc: 0.2778
Epoch 5/50
50000/50000 [==============================] - 3s 59us/step - loss: 6.0100 - acc: 0.2761 - val_loss: 5.9643 - val_acc: 0.2922
Epoch 6/50
50000/50000 [==============================] - 3s 57us/step - loss: 5.9741 - acc: 0.2883 - val_loss: 5.9335 - val_acc: 0.3038
Epoch 7/50
50000/50000 [==============================] - 3s 60us/step - loss: 5.9447 - acc: 0.3016 - val_loss: 5.9071 - val_acc